<a href="https://colab.research.google.com/github/tbnkh/home-page/blob/main/opus_mt_en_tn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using opus for TalkInSync TswanaMate
## installing some dependancies
transformers: to load in the model
sentencepiece: to tokenize the text
datasets: to use som data

In [2]:
!pip install transformers sentencepiece datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re
import string
import torch
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [4]:
english_path = "/content/drive/MyDrive/machine_learning/Autshumato.BilingualCorpus(English-Setswana).v2.0/Autshumato.BilingualCorpus(English-Setswana).v2.0/Autshumato.BilingualCorpus(English-Setswana).v2.0.en.txt"
setswana_path = "/content/drive/MyDrive/machine_learning/Autshumato.BilingualCorpus(English-Setswana).v2.0/Autshumato.BilingualCorpus(English-Setswana).v2.0/Autshumato.BilingualCorpus(English-Setswana).v2.0.tn.txt"


# Read English sentences
with open(english_path, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

# Read Setswana translations
with open(setswana_path, 'r', encoding='utf-8') as file:
    setswana_translations = file.readlines()

# Ensure the number of sentences match
assert len(english_sentences) == len(setswana_translations), "Number of sentences do not match"

# putting them together for easier handling
dataset = []

for english_sentence, setswana_translation in zip(english_sentences, setswana_translations):
    sample = {
        'english': english_sentence.strip(),
        'setswana': setswana_translation.strip()
    }
    dataset.append(sample)

# just to see our dataset
for sample in dataset[:5]:
    print(f"English: {sample['english']}")
    print(f"Setswana: {sample['setswana']}")
    print("="*30)

English: 6.7.10 Mechanisms to be established for referral of cases from traditional courts to magistrates ' courts and vice versa
Setswana: 6.7.10 Mekgwa e e tshwanetseng go tlhongwa fa go romelwa dikgetse go tswa kwa makgotleng a setso go ya kwa dikgotlatshekelo tsa bommagiseterata le go refosana 36
English: GCIS annual reports and other strategic documents .
Setswana: Pegelo ya ngwaga le ngwaga ya GCIS mmogo le ditokomane tse dingwe tsa ditogamaano .
English: Reduces soaking time
Setswana: O fokotsa nako ya go inela dilo
English: Have a look at the values !
Setswana: Lebang maitseo a lo tshwanetseng go nna le one !
English: Plan to write a poem .
Setswana: Rulaganya go kwala leboko .


In [5]:
from sklearn.model_selection import train_test_split

english_texts = [sample['english'] for sample in dataset]
setswana_texts = [sample['setswana'] for sample in dataset]

# Split the data
english_train, english_test, setswana_train, setswana_test = train_test_split(
    english_texts, setswana_texts, test_size=0.2, random_state=1)

# Create separate training and test datasets
train_dataset = [{'english': eng, 'setswana': setsw} for eng, setsw in zip(english_train, setswana_train)]
test_dataset = [{'english': eng, 'setswana': setsw} for eng, setsw in zip(english_test, setswana_test)]


In [6]:
train_dataset[0]

{'english': 'Uses the simple past tense',
 'setswana': 'Dirisa pakaphethi e e bonolo'}

Fine Tuning the model

In [7]:
# using pretrained model and then finetunig it on our dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-tn")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-tn").to('cuda')

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/828k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/296M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [8]:
optimizer = torch.optim.AdamW(model.parameters(),lr=0.0001)

In [9]:
def model_train(model, tokenizer, optimizer, X, y, max_epochs=15, n_batches=16):
    model.train()
    losses = 0
    for epoch in tqdm(range(max_epochs)):
        for i in tqdm(range(n_batches)):
            # making batches
            local_X, local_y = X[i * n_batches:(i + 1) * n_batches], y[i * n_batches:(i + 1) * n_batches]
            # preparing the data according to the model input
            batch = tokenizer.prepare_seq2seq_batch(list(local_X), list(local_y), return_tensors='pt').to('cuda')
            output = model(**batch)
            # loss can be taken directly from the model output
            loss = output.loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses = losses + loss

    return model

In [10]:
# Pass the model to the training function
model = model_train(model, tokenizer, optimizer, english_texts, setswana_texts)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3982: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is depr

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

...

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
a = model.generate(**tokenizer.prepare_seq2seq_batch(['I love dancing, what are your hobbies?'],return_tensors='pt').to('cuda'))
tokenizer.batch_decode(a)

['<pad> Ke rata go bina, ditiro tsa gago tsa go itumedisa ke dife?</s>']

In [13]:
torch.save(model , 'opus_mt_en_tn_fine_tune.pkl')
# Save the model
model.save_pretrained("/content/drive/MyDrive/machine_learning/opus_mt_en_tn_fine_tuned (1)")
